# Functional Programming in Scala Chapter 4

## Table of Contents
- [Option trait and Case Classes](#section0)
- [Exercise 4.1: Option Methods](#section1)
- [mean given code](#section1.5)
- [Exercise 4.2: variance](#section2)
- [Exercise 4.3: map2](#section3)
- [Exercise 4.4: list of Options](#section4)
- [Exercise 4.5: traverse](#section5)
- [Either trait and Case Classes](#section5.5)
- [Exercise 4.6: Either](#section6)
- [Exercise 4.7: Either sequence and traverse](#section7)
- [Exercise 4.8: question about given code](#section8)

<a id='section0'></a>

### Option trait and Case Classes

In [ ]:
trait Option[+A]{//define trait
    //all these functions are set up and implemented in 4.1
    def map[B](f: A=>B): Option[B] 
    def flatMap[B](f: A => Option[B]): Option[B] 
    def getOrElse[B >: A](default: => B): B 
    def orElse[B >: A](ob: => Option[B]): Option[B]
    def filter(f: A => Boolean): Option[A] 
}

case class Some[+A](get: A) extends Option[A]//case class for correct value
case object None extends Option[Nothing]// case class for incorrect value

<a id='section1'></a>

### Exercise 4.1: Option Methods
Implement all of the preceding functions on Option. As you implement each function, try to think about what it means and in what situations you’d use it. We’ll explore when to use each of these functions next. Here are a few hints for solving this exercise:

- It’s fine to use pattern matching, though you should be able to implement all the functions besides map and getOrElse without resorting to pattern matching.
- For map and flatMap, the type signature should be enough to determine the implementation.
- getOrElse returns the result inside the Some case of the Option, or if the Option is None, returns the given default value.
- orElse returns the first Option if it’s defined; otherwise, it returns the second Option.

In [23]:
trait Option[+A]{
    def map[B](f: A=>B): Option[B] = {
        //take the instance of Option apply f to its value and return a new Option
        this match {
            case None => None
            case Some(get) => Some(f(get))
        }
    }
    
    def flatMap[B](f: A => Option[B]): Option[B] = {
        //take the instance of Option apply f and return the new option
        map(f).getOrElse(None)
    }
    def getOrElse[B >: A](default: => B): B = {
        //return the value of this Option if it exists or return default value specified by user
        this match {
            case None => default
            case Some(get) => get
        }
    }
     def orElse[B >: A](ob: => Option[B]): Option[B] = {
         //return the option itself if not None or return value specified by user
         this match{
             case None => ob
             case _ => this
         }
     }
     def filter(f: A => Boolean): Option[A] = {
         //apply f to value of option 
         //if option is None return None
         // if f(value) if true return the option
         //if f(value) is false return None
         this match{
             case None => None
             case Some(get) if f(get) => this
             case Some(get) => None
         }
     }
}

case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

defined trait Option
defined class Some
defined object None

In [52]:
Some(1).map(_*2)
Some(1).flatMap(a => Some(2.0*a))

res22_0: Option[Int] = Some(2)
res22_1: Option[Double] = Some(2.0)

<a id='section1.5'></a>

### mean given code


In [1]:
//this function for mean is provided by the author and is used in 4.2
def mean(xs: Seq[Double]): Option[Double] =
//return the mean of a sequence
  if (xs.isEmpty) None//if sequence empty return None
  else Some(xs.sum / xs.length)//else return the result wrapped in option

defined function mean

<a id='section2'></a>

### Exercise 4.2: variance
Implement the variance function in terms of flatMap. If the mean of a sequence is m, the variance is the mean of math.pow(x - m, 2) for each element x in the sequence. See the definition of variance on Wikipedia (http://mng.bz/0Qsr).

In [46]:
def variance(xs: Seq[Double]): Option[Double] = {
    //calculate the variance of a given sequence
    mean(xs).flatMap(m => Some(xs.map(x=>math.pow(x-m,2)).sum/xs.length))
    //apply mean to the sequence if None then skip the rest of the computation else calculate the mean
}


defined function mean
defined function variance

In [50]:
variance(Seq(1,1,1,1,1))

res21: Option[Double] = Some(0.0)

<a id='section3'></a>

### Exercise 4.3: map2
Write a generic function map2 that combines two Option values using a binary function. If either Option value is None, then the return value is too. Here is its signature:

In [60]:
def map2[A,B,C](a: Option[A], b: Option[B])(f: (A,B) => C): Option[C] = {
    //accepts two options and a function that accepts two arguments of the Types wrapped in the two options
    //and returns the value of applying f to the two values in the Options passed, wrapped in an option
    a.flatMap(aa => b.map(bb => f(aa,bb)))
}

defined function map2

In [66]:
map2(Some(1),Some(2))((a,b) => a+b)

res18: Option[Int] = Some(3)

<a id='section4'></a>

### Exercise 4.4: list of Options
Write a function sequence that combines a list of Options into one Option containing a list of all the Some values in the original list. If the original list contains None even once, the result of the function should be None; otherwise the result should be Some with a list of all the values. Here is its signature:[3]

In [71]:
def sequence[A](a: List[Option[A]]): Option[List[A]] = {
    //iterate through a list of options and get each options value then convert list of values into option list of values
    Some(a.map(_.getOrElse(null.asInstanceOf[A])))
}

defined function sequence

In [72]:
sequence(List(Some(1), None, Some(4), None))

res20: Option[List[Int]] = Some(List(1, null, 4, null))

<a id='section5'></a>

### Exercise 4.5: traverse
Implement this function. It’s straightforward to do using map and sequence, but try for a more efficient implementation that only looks at the list once. In fact, implement sequence in terms of traverse.

In [73]:
def traverse[A,B](a: List[A])(f:A =>Option[B]): Option[List[B]] = {
    //iterate through each value of list apply f and get the value in the option then convert the resulting list
    //into an option list
    Some(a.map(i => f(i).getOrElse(null.asInstanceOf[B])))
}

def sequenceT[A](a: List[Option[A]]) : Option[List[A]] = {
    //same as sequence above but implemented using list
    traverse(a)(b => b)
}

defined function traverse

In [135]:
sequenceT(List(Some(1), None, Some(4), None))==sequence(List(Some(1), None, Some(4), None))

res44: Boolean = true

<a id='section5.5'></a>

### Either trait and Case Classes

In [82]:
//given code for Either trait and its two accompanying case classes
sealed trait Either[+E, +A]
case class Left[+E](value: E) extends Either[E, Nothing]//meant for exceptions
case class Right[+A](value: A) extends Either[Nothing, A]// meant for correct values

defined trait Either
defined class Left
defined class Right

<a id='section6'></a>

### Exercise 4.6: Either
Implement versions of map, flatMap, orElse, and map2 on Either that operate on the Right value.

In [98]:
trait Either[+E, +A]{
    def map[B](f: A => B): Either[E, B] = {
        //take instance of Either either return new exception if instance is an exception
        //or if its a correct value it applies f to value and returns result wrapped in Right
        this match {
            case Left(e) => Left(e)
            case Right(a) => Right(f(a))
        
        }
    }
    
    def flatMap[EE >: E, B](f: A => Either[EE,B]):Either[EE,B]={
        //take instance of Either either return new exception if instance is an exception
        //or if its a correct value it applies f to value and returns result 
        this match{
            case Left(e) => Left(e)
            case Right(a) => f(a)
        }
    }
    def orElse[EE >: E, B >: A](b: => Either[EE,B]): Either[EE, B] ={
        //take instance of Either either return default value chosen by user
        //or if its a correct value it returns a new Either with same value
        this match {
            case Left(e) => b
            case Right(a) => Right(a)
        }
    }
    
    def map2[EE >: E, B, C](b: Either[EE,B])(f: (A,B) => C) : Either[EE,C] ={
        //takes this instance and another instance of Either and a function that accepts two parameters
        //of the two types wrapped in each respective Either instance
        //applies f to the values of the two Eithers and returns the result wrapped in an Either
        this.flatMap(a => b.map(bb => f(a,bb)))
    }
}
case class Left[+E](value: E) extends Either[E, Nothing]
case class Right[+A](value: A) extends Either[Nothing, A]

defined trait Either
defined class Left
defined class Right

In [96]:
Right(0)map(1.0/_)
Right(0)flatMap(a => Right(1.0/a))

res33_0: Either[Nothing, Double] = Right(Infinity)
res33_1: Either[Nothing, Double] = Right(Infinity)

In [105]:
Right(1).map2(Right(4))((a,b) => a+b)

res36: Either[Nothing, Int] = Right(5)

<a id='section7'></a>

### Exercise 4.7: Either sequence and traverse
Implement sequence and traverse for Either. These should return the first error that’s encountered, if there is one.

In [133]:
def traverseE[E,A,B](es: List[A])(f: A => Either[E,B]): Either[E,List[B]] = {
    //same as traverse above except using Either instead of option
    Right(es.map(i => f(i) match {
        case Left(e) => null.asInstanceOf[B]
        case Right(a) => a
    }))
}

defined function traverseE

In [134]:
def sequenceET[E,A](es: List[Either[E,A]]):Either[E,List[A]] = {
    //same as sequence above except using Either instead of Option
    traverseE(es)(a => a)
}

defined function sequenceET

In [138]:
sequenceET(List(Right(1), Left("wrong"), Right(4), Left("wrong")))

res46: Either[String, List[Int]] = Right(List(1, null, 4, null))

<a id='section8'></a>

### Exercise 4.8: question about given code
In this implementation, map2 is only able to report one error, even if both the name and the age are invalid. What would you need to change in order to report both errors? Would you change map2 or the signature of mkPerson? Or could you create a new data type that captures this requirement better than Either does, with some additional structure? How would orElse, traverse, and sequence behave differently for that data type?

In [ ]:
//given code question 4.8 is refrencing
case class Person(name: Name, age: Age)
sealed class Name(val value: String)
sealed class Age(val value: Int)

def mkName(name: String): Either[String, Name] = 
  if (name == "" || name == null) Left("Name is empty.")
  else Right(new Name(name))

def mkAge(age: Int): Either[String, Age] =
  if (age < 0) Left("Age is out of range.")
  else Right(new Age(age))

def mkPerson(name: String, age: Int): Either[String, Person] =
  mkName(name).map2(mkAge(age))(Person(_, _))

** Answer:** I think a new data type would have to be created to handle the possibility of multiple errors.  I think for all three there would need to be additional case classes to allow for the multiple types of Exceptions that could have occured.